# lil_trees
Фундаментальная модель в ансамбле

Изучив специфику авторынка стало понятно, что бренд и модель - названия, не имеюще общих правил. Поэтому модель их может использовать только в качетсве dummy переменных. Но 500 с лишним категориальных редких переменных оказались слишком большим количеством для одной модели, предсказания портятся. Поэтому я решил разделить их специфику по моделям. Тут становится интереснее. Оказывается автомобили одной и той же модели, но разных комплектаций могут кратно отличаться в цене. Исходя из этого инсайта я понял, что можно сравнивать автомобили только в рамках одной модели. Этим и будет заниматься lil_trees

Оказалось, что в датасете множество машин одной и той же модели, более того, в тренировочном датасете оказалось 98% наименований моделей, покрывающие наименования тестового датасета. Исходя из этого, lil_trees будет иметь главный решающий голос в ансамбле в случаях, когда модель машины ему хорошо известна. Но это только при хорошей оптимизации.

Данная модель имеет недостаток - данных по одной модели может быть слишком мало, поэтому lil_trees работает в связке с [big_trees](./big_trees.ipynb), которая восполняет недостаток в данных.

## Data

In [1]:
import pandas as pd
import numpy as np

from sklearn import tree                             
from sklearn.model_selection import train_test_split

In [2]:
%store -r df

df = pd.concat(
[df[['brand','model_name','sell_id','price','engineDisplacement', 'enginePower','mileage','car_age', 'model_product_time']] ,
pd.get_dummies(df.bodyType,prefix='bodyType'),
pd.get_dummies(df.fuelType,prefix='fuelType'),
pd.get_dummies(df.vehicleTransmission,prefix='vehicleTransmission'),
pd.get_dummies(df.Привод,prefix='Привод'),
pd.get_dummies(df.Владельцы,prefix='Владельцы'),
pd.get_dummies(df.Владение,prefix='Владение'),
pd.get_dummies(df.color,prefix='color'),
pd.get_dummies(df.ПТС,prefix='ПТС'),
pd.get_dummies(df.Руль,prefix='Руль'),
pd.get_dummies(df.numberOfDoors,prefix='numberOfDoors'),],axis=1)

test = df[df.price==-1]
train = df[df.price!=-1]

## Func

In [3]:
'''Изменяет неуникальные названия моделей в соответствие брендам'''
def change_name_to_unique(df):
    df_non_unique = df[
        df.model_name.isin(valco_non_unique.index)]

    for i in df_non_unique.index:
        df.at[i,'model_name'] = \
        df.iloc[i].brand + '_' + df.iloc[i].model_name

Для этой модели потребуется сократить разновидности моделей в тренировочной выборки до соответствия их тестовой

## Предобработка

In [4]:
print(f'''{test.model_name.nunique()} - количество уникальных моделей 
{len(test.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - \
уникальны для всех предоставленных брендов''')

544 - количество уникальных моделей 
544 - количествоуникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - уникальны для всех предоставленных брендов


In [5]:
print(f'''{train.model_name.nunique()} - количество уникальных моделей 
{len(train.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название''')

1086 - количество уникальных моделей 
1105 - количествоуникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название


In [6]:
### Исключает модели, которых нет в тестовом датасете ###
filtered_train = train[train.model_name.isin(test.model_name.unique())
                      ].reset_index(drop=True)



'''  Делаем неуникальные для разных брендов названия моделей - уникальными  '''
###############################################################################
grouped_ft = filtered_train.groupby(['brand','model_name']).count()

valco_non_unique = grouped_ft.reset_index(level=1).model_name.value_counts()[
    grouped_ft.reset_index(level=1).model_name.value_counts()!=1]


change_name_to_unique(filtered_train)
change_name_to_unique(test)

### Удаляет модели в трейне, которых нет в тесте после уточнения модели брендом
models_to_drop = (set(filtered_train.model_name.unique()) -set(test.model_name.unique()))
index_to_drop = filtered_train[filtered_train.model_name.isin(models_to_drop)].index

filtered_train.drop(index_to_drop,inplace=True)
###############################################################################


In [7]:
unknown = list(set(test.model_name.unique())- set(filtered_train.model_name.unique()))
display(unknown)
print('18 Моделей из тестового датасета нет в тренировочном')

['ARTEON',
 'OTTI',
 'PRESEA',
 'POLO_GTI',
 'LANCER_RALLIART',
 'X4_M',
 'LM',
 'TORNEO',
 'RSQ3',
 'IST',
 'ZEST',
 'GOLF_R32',
 'SPARKY',
 'WILL_CYPHA',
 'PORTE',
 'Z',
 'COROLLA_II',
 '300ZX']

18 Моделей из тестового датасета нет в тренировочном


Модель будет специфичной под тренировочный датасет. Далее я разделяю тестовый датасет, но, важно, чтобы при разделении и соединении датасетов не потерялся порядок, в котором идут авто. 

## Модель

In [8]:
price = filtered_train[['price']]
for_train = filtered_train.drop(['brand','model_name','price','sell_id'],axis='columns')

for_pred = test.drop(['brand','model_name','price','sell_id'],axis='columns')

In [9]:
### Алгоритм разделения датасета и обучении деревьев по моделям авто
model_for_model = {}
for name in filtered_train.model_name.unique():
    model_for_model[name]=tree.DecisionTreeRegressor(
        max_depth=None, max_features=None)\
            .fit(for_train[filtered_train.model_name == name],
                 np.log(price[filtered_train.model_name == name]) )


In [10]:
#### АЛГОРИТМ ПРЕДСКАЗАНИЯ ~4 минуты
pred = []
for i in test.index:
    if test.model_name[i] in unknown: pred.append(-1)
    else: pred.append(model_for_model[test.model_name[i]]\
                      .predict((pd.DataFrame(for_pred.iloc[i]).T))[0])
        
print('KAGGLE score: 12.29984')

KAGGLE score: 12.29984


In [11]:
# сохранение результатов предскзания
pd.Series(np.exp(pred)).to_pickle('lil_pred.pickle') 

Сравнение моделей и использование lil_trees в стэкинге в следующем ноутбуке:
- [Models_and_stacking](./secondary_notebooks/Models_and_stacking.ipynb)